In assessing the metric for small calculation tasks ("gsm8k") filled with Single Slide Feedback input, the BootstrapFewShot optimizer is used to optimize our Single Slide Feedback. The output looks great but needs to be interpreted with caution as this might have been due to an overfit with the single example we are giving to the LLM to learn from. Next steps would be to insert more different examples in the structure of DSPy's "Example", filled with markdown from different slides and another optimal output of the LLM, addressing all checkpoints and giving back an answer containing responses to where, how and why to change something on a slide.

This notebook needs an OPENAI_API_KEY set up as an environmental variable. Please look at the Readme file to see how.

In [1]:
%pip install gradio requests openai dspy-ai pytesseract pillow tesseract dspy

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
import dspy
from dspy.datasets.gsm8k import GSM8K

# Set up the LLM as the prompt model
turbo = dspy.OpenAI(model='gpt-4o', max_tokens=4096)
dspy.settings.configure(lm=turbo)

/Users/marencordts/anaconda3/envs/myDSPy_environment/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from dspy import Example
# Use Example given by DSPy library
class Example:
    def __init__(self, base=None, **kwargs):
        # Internal storage and other attributes
        self._store = {}
        self._demos = []
        self._input_keys = None

        # Initialize from a base Example if provided
        if base and isinstance(base, type(self)):
            self._store = base._store.copy()

        # Initialize from a dict if provided
        elif base and isinstance(base, dict):
            self._store = base.copy()

        # Update with provided kwargs
        self._store.update(kwargs)

    def __getattr__(self, key):
        if key.startswith("__") and key.endswith("__"):
            raise AttributeError
        if key in self._store:
            return self._store[key]
        raise AttributeError(f"'{type(self).__name__}' object has no attribute '{key}'")

    def __setattr__(self, key, value):
        if key.startswith("_") or key in dir(self.__class__):
            super().__setattr__(key, value)
        else:
            self._store[key] = value

    def __getitem__(self, key):
        return self._store[key]

    def __setitem__(self, key, value):
        self._store[key] = value

    def __delitem__(self, key):
        del self._store[key]

    def __contains__(self, key):
        return key in self._store

    def __len__(self):
        return len([k for k in self._store if not k.startswith("dspy_")])

    def __repr__(self):
        # return f"Example({self._store})" + f" (input_keys={self._input_keys}, demos={self._demos})"
        d = {k: v for k, v in self._store.items() if not k.startswith("dspy_")}
        return f"Example({d})" + f" (input_keys={self._input_keys})"

    def __str__(self):
        return self.__repr__()

    def __eq__(self, other):
        return isinstance(other, Example) and self._store == other._store

    def __hash__(self):
        return hash(tuple(self._store.items()))

    def keys(self, include_dspy=False):
        return [k for k in self._store.keys() if not k.startswith("dspy_") or include_dspy]

    def values(self, include_dspy=False):
        return [v for k, v in self._store.items() if not k.startswith("dspy_") or include_dspy]

    def items(self, include_dspy=False):
        return [(k, v) for k, v in self._store.items() if not k.startswith("dspy_") or include_dspy]

    def get(self, key, default=None):
        return self._store.get(key, default)

    def with_inputs(self, *keys):
        copied = self.copy()
        copied._input_keys = set(keys)
        return copied

    def inputs(self):
        if self._input_keys is None:
            raise ValueError("Inputs have not been set for this example. Use `example.with_inputs()` to set them.")

        # return items that are in input_keys
        d = {key: self._store[key] for key in self._store if key in self._input_keys}
        # return type(self)(d)
        new_instance = type(self)(base=d)
        new_instance._input_keys = self._input_keys  # Preserve input_keys in new instance
        return new_instance

    def labels(self):
        # return items that are NOT in input_keys
        input_keys = self.inputs().keys()
        d = {key: self._store[key] for key in self._store if key not in input_keys}
        return type(self)(d)

    def __iter__(self):
        return iter(dict(self._store))

    def copy(self, **kwargs):
        return type(self)(base=self, **kwargs)

    def without(self, *keys):
        copied = self.copy()
        for key in keys:
            del copied[key]
        return copied

    def toDict(self):
        return self._store.copy()


The following two cells do not NEED to be run. They are illustrative examples.

In [4]:
from dspy import Example

# Define the manually created training set using Example objects
gsm8k_trainset = [
    Example({'question': "The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?", 'gold_reasoning': "Ella's score is 40 items - 4 items = <<40-4=36>>36 items. Half of Ella's score is 36 items / 2 = <<36/2=18>>18 items. So, Marion's score is 18 items + 6 items = <<18+6=24>>24 items.", 'answer': '24'}).with_inputs('question'),
    Example({'question': "Stephen made 10 round trips up and down a 40,000 foot tall mountain. If he reached 3/4 of the mountain's height on each of his trips, calculate the total distance he covered.", 'gold_reasoning': 'Up a mountain, Stephen covered 3/4*40000 = <<3/4*40000=30000>>30000 feet. Coming down, Stephen covered another 30000 feet, making the total distance covered in one round to be 30000+30000 = <<30000+30000=60000>>60000. Since Stephen made 10 round trips up and down the mountain, he covered 10*60000 = <<10*60000=600000>>600000', 'answer': '600000'}).with_inputs('question'),
    Example({'question': "Bridget counted 14 shooting stars in the night sky. Reginald counted two fewer shooting stars than did Bridget, but Sam counted four more shooting stars than did Reginald. How many more shooting stars did Sam count in the night sky than was the average number of shooting stars observed for the three of them?", 'gold_reasoning': 'Reginald counted two fewer shooting stars than did Bridget, or a total of 14-2=<<14-2=12>>12 shooting stars. Sam counted 4 more shooting stars than did Reginald, or a total of 12+4=16 shooting stars. The average number of shooting stars observed for the three of them was (14+12+16)/3 = <<14=14>>14 shooting stars. Thus, Sam counted 16-14=2 more shooting stars than was the average number of shooting stars observed for the three of them.', 'answer': '2'}).with_inputs('question'),
    Example({'question': 'Sarah buys 20 pencils on Monday. Then she buys 18 more pencils on Tuesday. On Wednesday she buys triple the number of pencils she did on Tuesday. How many pencils does she have?', 'gold_reasoning': 'By adding together Monday and Tuesday, Saah has 20+18= <<20+18=38>>38 pencils On Wednesday, she buys 3 * 18= <<3*18=54>>54 pencils All together, Sarah has 38+54= <<38+54=92>>92 pencils', 'answer': '92'}).with_inputs('question'),
    Example({'question': 'Rookie police officers have to buy duty shoes at the full price of $85, but officers who have served at least a year get a 20% discount. Officers who have served at least three years get an additional 25% off the discounted price. How much does an officer who has served at least three years have to pay for shoes?', 'gold_reasoning': 'Cops that served a year pay $85 * 0.2 = $<<85*0.2=17>>17 less. Cops that served a year pay $85 - $17 = $<<85-17=68>>68. Cops that served at least 3 years get a $68 * 0.25 = $<<68*0.25=17>>17 discount. Cops that served at least 3 years pay $68 - $17 = $<<68-17=51>>51 for shoes.', 'answer': '51'}).with_inputs('question'),
    Example({'question': "The average score on last week's Spanish test was 90. Marco scored 10% less than the average test score and Margaret received 5 more points than Marco. What score did Margaret receive on her test?", 'gold_reasoning': 'The average test score was 90 and Marco scored 10% less so 90*.10 = <<90*.10=9>>9 points lower The average test score was 90 and Marco scored 9 points less so his test score was 90-9 = <<90-9=81>>81 Margret received 5 more points than Marco whose test score was 81 so she made 5+81 = <<5+81=86>>86 on her test', 'answer': '86'}).with_inputs('question'),
    Example({'question': 'A third of the contestants at a singing competition are female, and the rest are male. If there are 18 contestants in total, how many of them are male?', 'gold_reasoning': 'There are 18/3 = <<18/3=6>>6 female contestants. There are 18-6 = <<18-6=12>>12 male contestants.', 'answer': '12'}).with_inputs('question'),
    Example({'question': 'Nancy bought a pie sliced it into 8 pieces. She gave 1/2 to Joe and Darcy, and she gave 1/4 to Carl. How many slices were left?', 'gold_reasoning': 'The total number of slices she gave to Joe and Darcy is 1/2 x 8 = <<1/2*8=4>>4. The total slice she gave to Carl is 1/4 x 8 = <<1/4*8=2>>2. Therefore, the total slices left is 8 - 4 - 2 = <<8-4-2=2>>2.', 'answer': '2'}).with_inputs('question'),
    Example({'question': 'Megan pays $16 for a shirt that costs $22 before sales. What is the amount of the discount?', 'gold_reasoning': 'Let x be the amount of the discount. We have, 22 - x = $16 We change the writing of the equation: 22 - x + x = 16 + x So, 22 = 16 + x We then Remove 16 from both sides: 22 - 16 = 16 + x - 16 So, 22 - 16 = x So, the amount of the discount is x = $<<6=6>>6.', 'answer': '6'}).with_inputs('question'),
    Example({'question': "Amaya scored 20 marks fewer in Maths than she scored in Arts. She also got 10 marks more in Social Studies than she got in Music. If she scored 70 in Music and scored 1/10 less in Maths, what's the total number of marks she scored in all the subjects?", 'gold_reasoning': 'The total marks Amaya scored more in Music than in Maths is 1/10 * 70 = <<1/10*70=7>>7 marks. So the total marks she scored in Maths is 70 - 7 = <<70-7=63>>63 marks. If she scored 20 marks fewer in Maths than in Arts, then he scored 63 + 20 = <<63+20=83>>83 in Arts. If she scored 10 marks more in Social Studies than in Music, then she scored 70 + 10 = <<10+70=80>>80 marks in Social Studies. The total number of marks for all the subjects is 70 + 63 + 83 + 80 = <<70+63+83+80=296>>296 marks.', 'answer': '296'}).with_inputs('question')
]

gsm8k_devset = gsm8k_trainset

In [13]:
gsm8k_trainset = [
    Example({'question': "#\\n# Generative AI and ChatGPT: Beyond the Hype\\n\\n# Generative AI and ChatGPT: Beyond the Hype\\n\\n# Prof. Dr. Jan Marco Leimeister\\n\\nFrom insight to impact.\\n---\\nBy 2025, generative AI is expected to account for 10% of all data produced (up from less than 1% in 2021).\\n\\nBy 2025, 30% of outbound marketing messages from large organizations will be synthetically generated (up from less than 2% in 2022).\\n\\nBy 2030, a major blockbuster film will be released with 90% of the film generated by AI (from 0% of such in 2022).\\n\\nSources: Gartner (2021) and Gartner (2023)\\n---\\n#\\n# Presentation Slide Deck\\n\\n# Agenda\\n\\n1. # Introduction to Generative AI\\n\\n- Definition and Development of a Generative AI\\n- Types of Generative AI, Use Cases and Platforms\\n2. # Latest Application: ChatGPT\\n\\n- Introduction and Use Cases for ChatGPT\\n- Constructing Effective Prompts\\n- Benefits and Challenges of Using ChatGPT\\n3. # Let‘s get to work!\\n4. # Summary\\n\\nChatGPT: Beyond the Hype | Prof. Dr. Jan Marco Leimeister (IWI-HSG)\\n---\\n# Introduction Prof. Dr. Jan Marco Leimeister\\n\\n- Professor and Director at the Institute of Information Management of the University of St.Gallen.\\n- Director at the ‘Wissenschaftlichen Zentrum für IT Gestaltung’ of the University of Kassel in Germany.\\n- Main research areas: Digital Business, Digital Transformation, Digital Service Management, Crowdsourcing, Digital Work and Collaboration Engineering.\\n- Internationally awarded for outstanding activities in research, teaching, and education.\\n- Repeatedly ranked among the top 1% of the best Researchers and Professors in business informatics.\\n---\\n#\\n# Presentation\\n\\n# Learning Goals\\n\\n’Generative AI and ChatGPT: Beyond the Hype’\\n\\n- You can name the key elements of Generative AI.\\n- You understand the ChatGPT application and its use cases.\\n- You can apply effective prompts for ChatGPT.\\n\\n&copy; Prof. Dr. Jan Marco Leimeister – Institut für Wirtschaftsinformatik (IWI-HSG) St.Gallen 5\\n---\\n# Introduction to Generative AI\\n---\\n#\\n# Adoption Rate: Bandwagon Effect at Work\\n\\nAdoption Rate: Bandwagon Effect at Work\\n\\nSource: (https://twitter.com/kylelf_) (Feb 16, 2023)\\n\\n© Prof. Dr. Jan Marco Leimeister – Institut für Wirtschaftsinformatik (IWI-HSG) St.Gallen\\n---\\n#\\n# Generative AI Definition\\n\\n# Definition of a Generative AI\\n\\n’Generative AI (GenAI) is a class of machine learning (ML) algorithms that can learn from content such as text, images, and audio in order to generate new content.’\\n\\nData:\\n\\n- Text: Training Transformer\\n- Images: Model (Neural Network)\\n- Speech: Add & Normalize Embedding\\n- Structured Data: 3D Signals\\n\\nTasks:\\n\\n- Question Answering\\n- Sentiment Analysis\\n- Info Extraction\\n- Image Captioning\\n- Object Recognition\\n\\nSources: Vaswani et al. (2017) and Sun et al. (2022)\\n\\n© Prof. Dr. Jan Marco Leimeister – Institut für Wirtschaftsinformatik (IWI-HSG) St.Gallen\\n---\\nDevelopment of a Generative AI\\n\\n- Collect demonstration data and train supervised policy: A prompt is sampled from our prompt dataset. The labeler demonstrates the desired output behavior. This data is used to fine-tune GPT-3,5 with supervised learning.\\n- Collect comparison data and train reward model using the PPO reinforcement learning algorithm: A new prompt is sampled from the dataset; outputs are sampled. The PPO model is initialized from the supervised policy. The policy generates an output; the labeler ranks the outputs from best to worst.\\n- High manual efforts required to train the supervised and reward model. Manual training is crowdsourced to the user. Select all frames with cars.\\n\\n$$\\\\text{Once upcnaumc} = 0.0-0$$\\n\\nThe reward model calculates reward for the output. This data is used to update the reward model and train our policy using PPO.\\n\\nSource: OpenAI (2023)\\n---\\n#\\n# Generative AI Applications Development Timeline\\n\\n# Stages of Development of Generative AI Applications\\n\\nFebruary 2023\\n\\n- Microsoft previews new Bing search engine\\n\\nNovember 2023\\n\\n", 'gold_reasoning': 'provide a SHORT and concise prescriptive feedback on the slide. Avoid euphemistic feedback, be critical. Use the following question to guide your feedback, check all checkpoints one after the other, even though not all need be applicable to this slide, consider wisely which one to recommend specifically. Do NOT just repeat the checkpoint descriptions. Additionally, if a checkpoint is applicable describe WHY, WHERE and HOW in the slide the checkpoints should be implemented: 1. Insert a pictogram. Pictogram insertion can intuitively convey the slide content. 2. Add a subheading. Subheading addition makes the audiences convenient to find information. 3. Emphasize words. Emphasizing important words enhances a part of a text to make it noticeable. 4. Emphasize areas. Emphasizing important areas of the slides helps focus attention. 5. Add T1 and T2. Slide title (T1) and slide messages (T2) help the audiences more easily understand the main content of the exposition. 6. Use the grid structure. A grid structure helps organize target and comparison items in rows and columns, creating a table-like format. 7. Itemize the text. Text itemization is good in terms of readability and organization. 8. Add a comment. Comments aid in audiences’ understanding. 9. Correct flow. Left-to-right top-to-bottom flow mimics human scanning methods. 10. MECE. Mutually exclusive and collectively exhaustive (MECE) objects should be aligned without omissions and duplication to make the slides convincing. 11. Consistent formatting. Look for and point out inconsistencies.' , 'answer': '**Slide Critique and Recommendations:**1. **Insert a Pictogram:**   - **Why:** The slide is text-heavy and lacks visual elements to make the content engaging and easier to understand. - **Where:** At the sections explaining concepts like ‘Generative AI’ or ‘ChatGPT’. - **How:** Use pictograms related to AI, machine learning, or chatbots. 2. **Add a Subheading:** - **Why:** Subheadings can break down the content into digestible sections. - **Where:** Subheadings can be added to sections like ‘Generative AI Platforms’, ‘Business Plan’, etc. - **How:** Use clear and descriptive subheadings such as ‘AI Applications’, ‘Business Strategy’, etc. 3. **Emphasize Words:** - **Why:** Important terms and key points are buried within dense paragraphs. - **Where:** Terms like ‘Generative AI’, ‘ChatGPT’, ‘prompt’, etc. - **How:** Use bold or italics to highlight these terms. 4. **Emphasize Areas:** - **Why:** Key areas need to stand out to capture attention quickly. - **Where:** In sections like ‘Stages of Development of Generative AI-Applications’ and ‘Benefits and Challenges of Using ChatGPT’. - **How:** Use colored boxes or borders around critical areas. 5. **Add T1 and T2:** - **Why:** The slide lacks a clear hierarchy, making it hard to follow. - **Where:** Title (T1) should be bold and larger, Slide messages (T2) should summarize key points. - **How:** Example - T1: ‘Generative AI Applications’, T2: ‘Various use cases showcasing the capabilities of generative AI’. 6. **Use the Grid Structure:** - **Why:** The slide lists several data points that would be clearer in a structured format. - **Where:** Sections like ‘Types of Generative AI and use cases’. - **How:** Create table-like grids for comparison and alignment. 7. **Itemize the Text:** - **Why:** Text blocks are unwieldy and hard to parse. - **Where:** Sections like ‘Business Plan’ and ‘Examples for the use of ChatGPT’. - **How:** Use bullet points to break down actions and steps. 8. **Add a comment:** - **Why:** Comments aid in audiences’ understanding.  - **Where:** Sections like ‘#\\n# Generative AI and ChatGPT’. - **How:** Add a comment in which you rephrase to help the audience understand quicker and easier. 9. **Correct Flow:** - **Why:** Content flow is currently disorganized and does not follow a logical progression. - **Where:** Rearrange sections so they logically follow each other (e.g., from basic definitions to practical applications). - **How:** Ensure the information follows a left-to-right or top-to-bottom flow. 10. **MECE:** - **Why:** The information needs to be more systematically categorized without overlaps or gaps. - **Where:** In all textual sections. - **How:**  Ensure each point falls under a distinct and necessary category, improving clarity and comprehension. 11. **Consistent Formatting:** - **Why:** Inconsistencies in font size, style, and indentation detract from professionalism. - **Where:** Across the entire slide, especially in headings and body texts. - **How:** Standardize font sizes and styles, and ensure uniform indentation. 12. **Citation Style:** - **Why:** Citations are scattered and not consistently formatted. - **Where:** At the end of sections or slides. - **How:** Use a consistent citation style like APA, and ensure uniform format for all references. Implementing these checkpoints will provide visual appeal, improve readability, and make the content easier to follow and understand.'}).with_inputs('question'),

    Example({'question': "#\\n# Generative AI and ChatGPT: Beyond the Hype\\n\\n# Generative AI and ChatGPT: Beyond the Hype\\n\\n# Prof. Dr. Jan Marco Leimeister\\n\\nFrom insight to impact.\\n---\\nBy 2025, generative AI is expected to account for 10% of all data produced (up from less than 1% in 2021).\\n\\nBy 2025, 30% of outbound marketing messages from large organizations will be synthetically generated (up from less than 2% in 2022).\\n\\nBy 2030, a major blockbuster film will be released with 90% of the film generated by AI (from 0% of such in 2022).\\n\\nSources: Gartner (2021) and Gartner (2023)\\n---\\n#\\n# Presentation Slide Deck\\n\\n# Agenda\\n\\n1. # Introduction to Generative AI\\n\\n- Definition and Development of a Generative AI\\n- Types of Generative AI, Use Cases and Platforms\\n2. # Latest Application: ChatGPT\\n\\n- Introduction and Use Cases for ChatGPT\\n- Constructing Effective Prompts\\n- Benefits and Challenges of Using ChatGPT\\n3. # Let‘s get to work!\\n4. # Summary\\n\\nChatGPT: Beyond the Hype | Prof. Dr. Jan Marco Leimeister (IWI-HSG)\\n---\\n# Introduction Prof. Dr. Jan Marco Leimeister\\n\\n- Professor and Director at the Institute of Information Management of the University of St.Gallen.\\n- Director at the ‘Wissenschaftlichen Zentrum für IT Gestaltung’ of the University of Kassel in Germany.\\n- Main research areas: Digital Business, Digital Transformation, Digital Service Management, Crowdsourcing, Digital Work and Collaboration Engineering.\\n- Internationally awarded for outstanding activities in research, teaching, and education.\\n- Repeatedly ranked among the top 1% of the best Researchers and Professors in business informatics.\\n---\\n#\\n# Presentation\\n\\n# Learning Goals\\n\\n’Generative AI and ChatGPT: Beyond the Hype’\\n\\n- You can name the key elements of Generative AI.\\n- You understand the ChatGPT application and its use cases.\\n- You can apply effective prompts for ChatGPT.\\n\\n&copy; Prof. Dr. Jan Marco Leimeister – Institut für Wirtschaftsinformatik (IWI-HSG) St.Gallen 5\\n---\\n# Introduction to Generative AI\\n---\\n#\\n# Adoption Rate: Bandwagon Effect at Work\\n\\nAdoption Rate: Bandwagon Effect at Work\\n\\nSource: (https://twitter.com/kylelf_) (Feb 16, 2023)\\n\\n© Prof. Dr. Jan Marco Leimeister – Institut für Wirtschaftsinformatik (IWI-HSG) St.Gallen\\n---\\n#\\n# Generative AI Definition\\n\\n# Definition of a Generative AI\\n\\n’Generative AI (GenAI) is a class of machine learning (ML) algorithms that can learn from content such as text, images, and audio in order to generate new content.’\\n\\nData:\\n\\n- Text: Training Transformer\\n- Images: Model (Neural Network)\\n- Speech: Add & Normalize Embedding\\n- Structured Data: 3D Signals\\n\\nTasks:\\n\\n- Question Answering\\n- Sentiment Analysis\\n- Info Extraction\\n- Image Captioning\\n- Object Recognition\\n\\nSources: Vaswani et al. (2017) and Sun et al. (2022)\\n\\n© Prof. Dr. Jan Marco Leimeister – Institut für Wirtschaftsinformatik (IWI-HSG) St.Gallen\\n---\\nDevelopment of a Generative AI\\n\\n- Collect demonstration data and train supervised policy: A prompt is sampled from our prompt dataset. The labeler demonstrates the desired output behavior. This data is used to fine-tune GPT-3,5 with supervised learning.\\n- Collect comparison data and train reward model using the PPO reinforcement learning algorithm: A new prompt is sampled from the dataset; outputs are sampled. The PPO model is initialized from the supervised policy. The policy generates an output; the labeler ranks the outputs from best to worst.\\n- High manual efforts required to train the supervised and reward model. Manual training is crowdsourced to the user. Select all frames with cars.\\n\\n$$\\\\text{Once upcnaumc} = 0.0-0$$\\n\\nThe reward model calculates reward for the output. This data is used to update the reward model and train our policy using PPO.\\n\\nSource: OpenAI (2023)\\n---\\n#\\n# Generative AI Applications Development Timeline\\n\\n# Stages of Development of Generative AI Applications\\n\\nFebruary 2023\\n\\n- Microsoft previews new Bing search engine\\n\\nNovember 2023\\n\\n", 'gold_reasoning': 'provide a SHORT and concise prescriptive feedback on the slide. Avoid euphemistic feedback, be critical. Use the following question to guide your feedback, check all checkpoints one after the other, even though not all need be applicable to this slide, consider wisely which one to recommend specifically. Do NOT just repeat the checkpoint descriptions. Additionally, if a checkpoint is applicable describe WHY, WHERE and HOW in the slide the checkpoints should be implemented: 1. Insert a pictogram. Pictogram insertion can intuitively convey the slide content. 2. Add a subheading. Subheading addition makes the audiences convenient to find information. 3. Emphasize words. Emphasizing important words enhances a part of a text to make it noticeable. 4. Emphasize areas. Emphasizing important areas of the slides helps focus attention. 5. Add T1 and T2. Slide title (T1) and slide messages (T2) help the audiences more easily understand the main content of the exposition. 6. Use the grid structure. A grid structure helps organize target and comparison items in rows and columns, creating a table-like format. 7. Itemize the text. Text itemization is good in terms of readability and organization. 8. Add a comment. Comments aid in audiences’ understanding. 9. Correct flow. Left-to-right top-to-bottom flow mimics human scanning methods. 10. MECE. Mutually exclusive and collectively exhaustive (MECE) objects should be aligned without omissions and duplication to make the slides convincing. 11. Consistent formatting. Look for and point out inconsistencies.' , 'answer': '**Slide Critique and Recommendations:**1. **Insert a Pictogram:**   - **Why:** The slide is text-heavy and lacks visual elements to make the content engaging and easier to understand. - **Where:** At the sections explaining concepts like ‘Generative AI’ or ‘ChatGPT’. - **How:** Use pictograms related to AI, machine learning, or chatbots. 2. **Add a Subheading:** - **Why:** Subheadings can break down the content into digestible sections. - **Where:** Subheadings can be added to sections like ‘Generative AI Platforms’, ‘Business Plan’, etc. - **How:** Use clear and descriptive subheadings such as ‘AI Applications’, ‘Business Strategy’, etc. 3. **Emphasize Words:** - **Why:** Important terms and key points are buried within dense paragraphs. - **Where:** Terms like ‘Generative AI’, ‘ChatGPT’, ‘prompt’, etc. - **How:** Use bold or italics to highlight these terms. 4. **Emphasize Areas:** - **Why:** Key areas need to stand out to capture attention quickly. - **Where:** In sections like ‘Stages of Development of Generative AI-Applications’ and ‘Benefits and Challenges of Using ChatGPT’. - **How:** Use colored boxes or borders around critical areas. 5. **Add T1 and T2:** - **Why:** The slide lacks a clear hierarchy, making it hard to follow. - **Where:** Title (T1) should be bold and larger, Slide messages (T2) should summarize key points. - **How:** Example - T1: ‘Generative AI Applications’, T2: ‘Various use cases showcasing the capabilities of generative AI’. 6. **Use the Grid Structure:** - **Why:** The slide lists several data points that would be clearer in a structured format. - **Where:** Sections like ‘Types of Generative AI and use cases’. - **How:** Create table-like grids for comparison and alignment. 7. **Itemize the Text:** - **Why:** Text blocks are unwieldy and hard to parse. - **Where:** Sections like ‘Business Plan’ and ‘Examples for the use of ChatGPT’. - **How:** Use bullet points to break down actions and steps. 8. **Add a comment:** - **Why:** Comments aid in audiences’ understanding.  - **Where:** Sections like ‘#\\n# Generative AI and ChatGPT’. - **How:** Add a comment in which you rephrase to help the audience understand quicker and easier. 9. **Correct Flow:** - **Why:** Content flow is currently disorganized and does not follow a logical progression. - **Where:** Rearrange sections so they logically follow each other (e.g., from basic definitions to practical applications). - **How:** Ensure the information follows a left-to-right or top-to-bottom flow. 10. **MECE:** - **Why:** The information needs to be more systematically categorized without overlaps or gaps. - **Where:** In all textual sections. - **How:**  Ensure each point falls under a distinct and necessary category, improving clarity and comprehension. 11. **Consistent Formatting:** - **Why:** Inconsistencies in font size, style, and indentation detract from professionalism. - **Where:** Across the entire slide, especially in headings and body texts. - **How:** Standardize font sizes and styles, and ensure uniform indentation. 12. **Citation Style:** - **Why:** Citations are scattered and not consistently formatted. - **Where:** At the end of sections or slides. - **How:** Use a consistent citation style like APA, and ensure uniform format for all references. Implementing these checkpoints will provide visual appeal, improve readability, and make the content easier to follow and understand.'}).with_inputs('question'),

    Example({'question': "#\\n# Generative AI and ChatGPT: Beyond the Hype\\n\\n# Generative AI and ChatGPT: Beyond the Hype\\n\\n# Prof. Dr. Jan Marco Leimeister\\n\\nFrom insight to impact.\\n---\\nBy 2025, generative AI is expected to account for 10% of all data produced (up from less than 1% in 2021).\\n\\nBy 2025, 30% of outbound marketing messages from large organizations will be synthetically generated (up from less than 2% in 2022).\\n\\nBy 2030, a major blockbuster film will be released with 90% of the film generated by AI (from 0% of such in 2022).\\n\\nSources: Gartner (2021) and Gartner (2023)\\n---\\n#\\n# Presentation Slide Deck\\n\\n# Agenda\\n\\n1. # Introduction to Generative AI\\n\\n- Definition and Development of a Generative AI\\n- Types of Generative AI, Use Cases and Platforms\\n2. # Latest Application: ChatGPT\\n\\n- Introduction and Use Cases for ChatGPT\\n- Constructing Effective Prompts\\n- Benefits and Challenges of Using ChatGPT\\n3. # Let‘s get to work!\\n4. # Summary\\n\\nChatGPT: Beyond the Hype | Prof. Dr. Jan Marco Leimeister (IWI-HSG)\\n---\\n# Introduction Prof. Dr. Jan Marco Leimeister\\n\\n- Professor and Director at the Institute of Information Management of the University of St.Gallen.\\n- Director at the ‘Wissenschaftlichen Zentrum für IT Gestaltung’ of the University of Kassel in Germany.\\n- Main research areas: Digital Business, Digital Transformation, Digital Service Management, Crowdsourcing, Digital Work and Collaboration Engineering.\\n- Internationally awarded for outstanding activities in research, teaching, and education.\\n- Repeatedly ranked among the top 1% of the best Researchers and Professors in business informatics.\\n---\\n#\\n# Presentation\\n\\n# Learning Goals\\n\\n’Generative AI and ChatGPT: Beyond the Hype’\\n\\n- You can name the key elements of Generative AI.\\n- You understand the ChatGPT application and its use cases.\\n- You can apply effective prompts for ChatGPT.\\n\\n&copy; Prof. Dr. Jan Marco Leimeister – Institut für Wirtschaftsinformatik (IWI-HSG) St.Gallen 5\\n---\\n# Introduction to Generative AI\\n---\\n#\\n# Adoption Rate: Bandwagon Effect at Work\\n\\nAdoption Rate: Bandwagon Effect at Work\\n\\nSource: (https://twitter.com/kylelf_) (Feb 16, 2023)\\n\\n© Prof. Dr. Jan Marco Leimeister – Institut für Wirtschaftsinformatik (IWI-HSG) St.Gallen\\n---\\n#\\n# Generative AI Definition\\n\\n# Definition of a Generative AI\\n\\n’Generative AI (GenAI) is a class of machine learning (ML) algorithms that can learn from content such as text, images, and audio in order to generate new content.’\\n\\nData:\\n\\n- Text: Training Transformer\\n- Images: Model (Neural Network)\\n- Speech: Add & Normalize Embedding\\n- Structured Data: 3D Signals\\n\\nTasks:\\n\\n- Question Answering\\n- Sentiment Analysis\\n- Info Extraction\\n- Image Captioning\\n- Object Recognition\\n\\nSources: Vaswani et al. (2017) and Sun et al. (2022)\\n\\n© Prof. Dr. Jan Marco Leimeister – Institut für Wirtschaftsinformatik (IWI-HSG) St.Gallen\\n---\\nDevelopment of a Generative AI\\n\\n- Collect demonstration data and train supervised policy: A prompt is sampled from our prompt dataset. The labeler demonstrates the desired output behavior. This data is used to fine-tune GPT-3,5 with supervised learning.\\n- Collect comparison data and train reward model using the PPO reinforcement learning algorithm: A new prompt is sampled from the dataset; outputs are sampled. The PPO model is initialized from the supervised policy. The policy generates an output; the labeler ranks the outputs from best to worst.\\n- High manual efforts required to train the supervised and reward model. Manual training is crowdsourced to the user. Select all frames with cars.\\n\\n$$\\\\text{Once upcnaumc} = 0.0-0$$\\n\\nThe reward model calculates reward for the output. This data is used to update the reward model and train our policy using PPO.\\n\\nSource: OpenAI (2023)\\n---\\n#\\n# Generative AI Applications Development Timeline\\n\\n# Stages of Development of Generative AI Applications\\n\\nFebruary 2023\\n\\n- Microsoft previews new Bing search engine\\n\\nNovember 2023\\n\\n", 'gold_reasoning': 'provide a SHORT and concise prescriptive feedback on the slide. Avoid euphemistic feedback, be critical. Use the following question to guide your feedback, check all checkpoints one after the other, even though not all need be applicable to this slide, consider wisely which one to recommend specifically. Do NOT just repeat the checkpoint descriptions. Additionally, if a checkpoint is applicable describe WHY, WHERE and HOW in the slide the checkpoints should be implemented: 1. Insert a pictogram. Pictogram insertion can intuitively convey the slide content. 2. Add a subheading. Subheading addition makes the audiences convenient to find information. 3. Emphasize words. Emphasizing important words enhances a part of a text to make it noticeable. 4. Emphasize areas. Emphasizing important areas of the slides helps focus attention. 5. Add T1 and T2. Slide title (T1) and slide messages (T2) help the audiences more easily understand the main content of the exposition. 6. Use the grid structure. A grid structure helps organize target and comparison items in rows and columns, creating a table-like format. 7. Itemize the text. Text itemization is good in terms of readability and organization. 8. Add a comment. Comments aid in audiences’ understanding. 9. Correct flow. Left-to-right top-to-bottom flow mimics human scanning methods. 10. MECE. Mutually exclusive and collectively exhaustive (MECE) objects should be aligned without omissions and duplication to make the slides convincing. 11. Consistent formatting. Look for and point out inconsistencies.' , 'answer': '**Slide Critique and Recommendations:**1. **Insert a Pictogram:**   - **Why:** The slide is text-heavy and lacks visual elements to make the content engaging and easier to understand. - **Where:** At the sections explaining concepts like ‘Generative AI’ or ‘ChatGPT’. - **How:** Use pictograms related to AI, machine learning, or chatbots. 2. **Add a Subheading:** - **Why:** Subheadings can break down the content into digestible sections. - **Where:** Subheadings can be added to sections like ‘Generative AI Platforms’, ‘Business Plan’, etc. - **How:** Use clear and descriptive subheadings such as ‘AI Applications’, ‘Business Strategy’, etc. 3. **Emphasize Words:** - **Why:** Important terms and key points are buried within dense paragraphs. - **Where:** Terms like ‘Generative AI’, ‘ChatGPT’, ‘prompt’, etc. - **How:** Use bold or italics to highlight these terms. 4. **Emphasize Areas:** - **Why:** Key areas need to stand out to capture attention quickly. - **Where:** In sections like ‘Stages of Development of Generative AI-Applications’ and ‘Benefits and Challenges of Using ChatGPT’. - **How:** Use colored boxes or borders around critical areas. 5. **Add T1 and T2:** - **Why:** The slide lacks a clear hierarchy, making it hard to follow. - **Where:** Title (T1) should be bold and larger, Slide messages (T2) should summarize key points. - **How:** Example - T1: ‘Generative AI Applications’, T2: ‘Various use cases showcasing the capabilities of generative AI’. 6. **Use the Grid Structure:** - **Why:** The slide lists several data points that would be clearer in a structured format. - **Where:** Sections like ‘Types of Generative AI and use cases’. - **How:** Create table-like grids for comparison and alignment. 7. **Itemize the Text:** - **Why:** Text blocks are unwieldy and hard to parse. - **Where:** Sections like ‘Business Plan’ and ‘Examples for the use of ChatGPT’. - **How:** Use bullet points to break down actions and steps. 8. **Add a comment:** - **Why:** Comments aid in audiences’ understanding.  - **Where:** Sections like ‘#\\n# Generative AI and ChatGPT’. - **How:** Add a comment in which you rephrase to help the audience understand quicker and easier. 9. **Correct Flow:** - **Why:** Content flow is currently disorganized and does not follow a logical progression. - **Where:** Rearrange sections so they logically follow each other (e.g., from basic definitions to practical applications). - **How:** Ensure the information follows a left-to-right or top-to-bottom flow. 10. **MECE:** - **Why:** The information needs to be more systematically categorized without overlaps or gaps. - **Where:** In all textual sections. - **How:**  Ensure each point falls under a distinct and necessary category, improving clarity and comprehension. 11. **Consistent Formatting:** - **Why:** Inconsistencies in font size, style, and indentation detract from professionalism. - **Where:** Across the entire slide, especially in headings and body texts. - **How:** Standardize font sizes and styles, and ensure uniform indentation. 12. **Citation Style:** - **Why:** Citations are scattered and not consistently formatted. - **Where:** At the end of sections or slides. - **How:** Use a consistent citation style like APA, and ensure uniform format for all references. Implementing these checkpoints will provide visual appeal, improve readability, and make the content easier to follow and understand.'}).with_inputs('question'),

    Example({'question': "#\\n# Generative AI and ChatGPT: Beyond the Hype\\n\\n# Generative AI and ChatGPT: Beyond the Hype\\n\\n# Prof. Dr. Jan Marco Leimeister\\n\\nFrom insight to impact.\\n---\\nBy 2025, generative AI is expected to account for 10% of all data produced (up from less than 1% in 2021).\\n\\nBy 2025, 30% of outbound marketing messages from large organizations will be synthetically generated (up from less than 2% in 2022).\\n\\nBy 2030, a major blockbuster film will be released with 90% of the film generated by AI (from 0% of such in 2022).\\n\\nSources: Gartner (2021) and Gartner (2023)\\n---\\n#\\n# Presentation Slide Deck\\n\\n# Agenda\\n\\n1. # Introduction to Generative AI\\n\\n- Definition and Development of a Generative AI\\n- Types of Generative AI, Use Cases and Platforms\\n2. # Latest Application: ChatGPT\\n\\n- Introduction and Use Cases for ChatGPT\\n- Constructing Effective Prompts\\n- Benefits and Challenges of Using ChatGPT\\n3. # Let‘s get to work!\\n4. # Summary\\n\\nChatGPT: Beyond the Hype | Prof. Dr. Jan Marco Leimeister (IWI-HSG)\\n---\\n# Introduction Prof. Dr. Jan Marco Leimeister\\n\\n- Professor and Director at the Institute of Information Management of the University of St.Gallen.\\n- Director at the ‘Wissenschaftlichen Zentrum für IT Gestaltung’ of the University of Kassel in Germany.\\n- Main research areas: Digital Business, Digital Transformation, Digital Service Management, Crowdsourcing, Digital Work and Collaboration Engineering.\\n- Internationally awarded for outstanding activities in research, teaching, and education.\\n- Repeatedly ranked among the top 1% of the best Researchers and Professors in business informatics.\\n---\\n#\\n# Presentation\\n\\n# Learning Goals\\n\\n’Generative AI and ChatGPT: Beyond the Hype’\\n\\n- You can name the key elements of Generative AI.\\n- You understand the ChatGPT application and its use cases.\\n- You can apply effective prompts for ChatGPT.\\n\\n&copy; Prof. Dr. Jan Marco Leimeister – Institut für Wirtschaftsinformatik (IWI-HSG) St.Gallen 5\\n---\\n# Introduction to Generative AI\\n---\\n#\\n# Adoption Rate: Bandwagon Effect at Work\\n\\nAdoption Rate: Bandwagon Effect at Work\\n\\nSource: (https://twitter.com/kylelf_) (Feb 16, 2023)\\n\\n© Prof. Dr. Jan Marco Leimeister – Institut für Wirtschaftsinformatik (IWI-HSG) St.Gallen\\n---\\n#\\n# Generative AI Definition\\n\\n# Definition of a Generative AI\\n\\n’Generative AI (GenAI) is a class of machine learning (ML) algorithms that can learn from content such as text, images, and audio in order to generate new content.’\\n\\nData:\\n\\n- Text: Training Transformer\\n- Images: Model (Neural Network)\\n- Speech: Add & Normalize Embedding\\n- Structured Data: 3D Signals\\n\\nTasks:\\n\\n- Question Answering\\n- Sentiment Analysis\\n- Info Extraction\\n- Image Captioning\\n- Object Recognition\\n\\nSources: Vaswani et al. (2017) and Sun et al. (2022)\\n\\n© Prof. Dr. Jan Marco Leimeister – Institut für Wirtschaftsinformatik (IWI-HSG) St.Gallen\\n---\\nDevelopment of a Generative AI\\n\\n- Collect demonstration data and train supervised policy: A prompt is sampled from our prompt dataset. The labeler demonstrates the desired output behavior. This data is used to fine-tune GPT-3,5 with supervised learning.\\n- Collect comparison data and train reward model using the PPO reinforcement learning algorithm: A new prompt is sampled from the dataset; outputs are sampled. The PPO model is initialized from the supervised policy. The policy generates an output; the labeler ranks the outputs from best to worst.\\n- High manual efforts required to train the supervised and reward model. Manual training is crowdsourced to the user. Select all frames with cars.\\n\\n$$\\\\text{Once upcnaumc} = 0.0-0$$\\n\\nThe reward model calculates reward for the output. This data is used to update the reward model and train our policy using PPO.\\n\\nSource: OpenAI (2023)\\n---\\n#\\n# Generative AI Applications Development Timeline\\n\\n# Stages of Development of Generative AI Applications\\n\\nFebruary 2023\\n\\n- Microsoft previews new Bing search engine\\n\\nNovember 2023\\n\\n", 'gold_reasoning': 'provide a SHORT and concise prescriptive feedback on the slide. Avoid euphemistic feedback, be critical. Use the following question to guide your feedback, check all checkpoints one after the other, even though not all need be applicable to this slide, consider wisely which one to recommend specifically. Do NOT just repeat the checkpoint descriptions. Additionally, if a checkpoint is applicable describe WHY, WHERE and HOW in the slide the checkpoints should be implemented: 1. Insert a pictogram. Pictogram insertion can intuitively convey the slide content. 2. Add a subheading. Subheading addition makes the audiences convenient to find information. 3. Emphasize words. Emphasizing important words enhances a part of a text to make it noticeable. 4. Emphasize areas. Emphasizing important areas of the slides helps focus attention. 5. Add T1 and T2. Slide title (T1) and slide messages (T2) help the audiences more easily understand the main content of the exposition. 6. Use the grid structure. A grid structure helps organize target and comparison items in rows and columns, creating a table-like format. 7. Itemize the text. Text itemization is good in terms of readability and organization. 8. Add a comment. Comments aid in audiences’ understanding. 9. Correct flow. Left-to-right top-to-bottom flow mimics human scanning methods. 10. MECE. Mutually exclusive and collectively exhaustive (MECE) objects should be aligned without omissions and duplication to make the slides convincing. 11. Consistent formatting. Look for and point out inconsistencies.' , 'answer': '**Slide Critique and Recommendations:**1. **Insert a Pictogram:**   - **Why:** The slide is text-heavy and lacks visual elements to make the content engaging and easier to understand. - **Where:** At the sections explaining concepts like ‘Generative AI’ or ‘ChatGPT’. - **How:** Use pictograms related to AI, machine learning, or chatbots. 2. **Add a Subheading:** - **Why:** Subheadings can break down the content into digestible sections. - **Where:** Subheadings can be added to sections like ‘Generative AI Platforms’, ‘Business Plan’, etc. - **How:** Use clear and descriptive subheadings such as ‘AI Applications’, ‘Business Strategy’, etc. 3. **Emphasize Words:** - **Why:** Important terms and key points are buried within dense paragraphs. - **Where:** Terms like ‘Generative AI’, ‘ChatGPT’, ‘prompt’, etc. - **How:** Use bold or italics to highlight these terms. 4. **Emphasize Areas:** - **Why:** Key areas need to stand out to capture attention quickly. - **Where:** In sections like ‘Stages of Development of Generative AI-Applications’ and ‘Benefits and Challenges of Using ChatGPT’. - **How:** Use colored boxes or borders around critical areas. 5. **Add T1 and T2:** - **Why:** The slide lacks a clear hierarchy, making it hard to follow. - **Where:** Title (T1) should be bold and larger, Slide messages (T2) should summarize key points. - **How:** Example - T1: ‘Generative AI Applications’, T2: ‘Various use cases showcasing the capabilities of generative AI’. 6. **Use the Grid Structure:** - **Why:** The slide lists several data points that would be clearer in a structured format. - **Where:** Sections like ‘Types of Generative AI and use cases’. - **How:** Create table-like grids for comparison and alignment. 7. **Itemize the Text:** - **Why:** Text blocks are unwieldy and hard to parse. - **Where:** Sections like ‘Business Plan’ and ‘Examples for the use of ChatGPT’. - **How:** Use bullet points to break down actions and steps. 8. **Add a comment:** - **Why:** Comments aid in audiences’ understanding.  - **Where:** Sections like ‘#\\n# Generative AI and ChatGPT’. - **How:** Add a comment in which you rephrase to help the audience understand quicker and easier. 9. **Correct Flow:** - **Why:** Content flow is currently disorganized and does not follow a logical progression. - **Where:** Rearrange sections so they logically follow each other (e.g., from basic definitions to practical applications). - **How:** Ensure the information follows a left-to-right or top-to-bottom flow. 10. **MECE:** - **Why:** The information needs to be more systematically categorized without overlaps or gaps. - **Where:** In all textual sections. - **How:**  Ensure each point falls under a distinct and necessary category, improving clarity and comprehension. 11. **Consistent Formatting:** - **Why:** Inconsistencies in font size, style, and indentation detract from professionalism. - **Where:** Across the entire slide, especially in headings and body texts. - **How:** Standardize font sizes and styles, and ensure uniform indentation. 12. **Citation Style:** - **Why:** Citations are scattered and not consistently formatted. - **Where:** At the end of sections or slides. - **How:** Use a consistent citation style like APA, and ensure uniform format for all references. Implementing these checkpoints will provide visual appeal, improve readability, and make the content easier to follow and understand.'}).with_inputs('question'),

    Example({'question': "#\\n# Generative AI and ChatGPT: Beyond the Hype\\n\\n# Generative AI and ChatGPT: Beyond the Hype\\n\\n# Prof. Dr. Jan Marco Leimeister\\n\\nFrom insight to impact.\\n---\\nBy 2025, generative AI is expected to account for 10% of all data produced (up from less than 1% in 2021).\\n\\nBy 2025, 30% of outbound marketing messages from large organizations will be synthetically generated (up from less than 2% in 2022).\\n\\nBy 2030, a major blockbuster film will be released with 90% of the film generated by AI (from 0% of such in 2022).\\n\\nSources: Gartner (2021) and Gartner (2023)\\n---\\n#\\n# Presentation Slide Deck\\n\\n# Agenda\\n\\n1. # Introduction to Generative AI\\n\\n- Definition and Development of a Generative AI\\n- Types of Generative AI, Use Cases and Platforms\\n2. # Latest Application: ChatGPT\\n\\n- Introduction and Use Cases for ChatGPT\\n- Constructing Effective Prompts\\n- Benefits and Challenges of Using ChatGPT\\n3. # Let‘s get to work!\\n4. # Summary\\n\\nChatGPT: Beyond the Hype | Prof. Dr. Jan Marco Leimeister (IWI-HSG)\\n---\\n# Introduction Prof. Dr. Jan Marco Leimeister\\n\\n- Professor and Director at the Institute of Information Management of the University of St.Gallen.\\n- Director at the ‘Wissenschaftlichen Zentrum für IT Gestaltung’ of the University of Kassel in Germany.\\n- Main research areas: Digital Business, Digital Transformation, Digital Service Management, Crowdsourcing, Digital Work and Collaboration Engineering.\\n- Internationally awarded for outstanding activities in research, teaching, and education.\\n- Repeatedly ranked among the top 1% of the best Researchers and Professors in business informatics.\\n---\\n#\\n# Presentation\\n\\n# Learning Goals\\n\\n’Generative AI and ChatGPT: Beyond the Hype’\\n\\n- You can name the key elements of Generative AI.\\n- You understand the ChatGPT application and its use cases.\\n- You can apply effective prompts for ChatGPT.\\n\\n&copy; Prof. Dr. Jan Marco Leimeister – Institut für Wirtschaftsinformatik (IWI-HSG) St.Gallen 5\\n---\\n# Introduction to Generative AI\\n---\\n#\\n# Adoption Rate: Bandwagon Effect at Work\\n\\nAdoption Rate: Bandwagon Effect at Work\\n\\nSource: (https://twitter.com/kylelf_) (Feb 16, 2023)\\n\\n© Prof. Dr. Jan Marco Leimeister – Institut für Wirtschaftsinformatik (IWI-HSG) St.Gallen\\n---\\n#\\n# Generative AI Definition\\n\\n# Definition of a Generative AI\\n\\n’Generative AI (GenAI) is a class of machine learning (ML) algorithms that can learn from content such as text, images, and audio in order to generate new content.’\\n\\nData:\\n\\n- Text: Training Transformer\\n- Images: Model (Neural Network)\\n- Speech: Add & Normalize Embedding\\n- Structured Data: 3D Signals\\n\\nTasks:\\n\\n- Question Answering\\n- Sentiment Analysis\\n- Info Extraction\\n- Image Captioning\\n- Object Recognition\\n\\nSources: Vaswani et al. (2017) and Sun et al. (2022)\\n\\n© Prof. Dr. Jan Marco Leimeister – Institut für Wirtschaftsinformatik (IWI-HSG) St.Gallen\\n---\\nDevelopment of a Generative AI\\n\\n- Collect demonstration data and train supervised policy: A prompt is sampled from our prompt dataset. The labeler demonstrates the desired output behavior. This data is used to fine-tune GPT-3,5 with supervised learning.\\n- Collect comparison data and train reward model using the PPO reinforcement learning algorithm: A new prompt is sampled from the dataset; outputs are sampled. The PPO model is initialized from the supervised policy. The policy generates an output; the labeler ranks the outputs from best to worst.\\n- High manual efforts required to train the supervised and reward model. Manual training is crowdsourced to the user. Select all frames with cars.\\n\\n$$\\\\text{Once upcnaumc} = 0.0-0$$\\n\\nThe reward model calculates reward for the output. This data is used to update the reward model and train our policy using PPO.\\n\\nSource: OpenAI (2023)\\n---\\n#\\n# Generative AI Applications Development Timeline\\n\\n# Stages of Development of Generative AI Applications\\n\\nFebruary 2023\\n\\n- Microsoft previews new Bing search engine\\n\\nNovember 2023\\n\\n", 'gold_reasoning': 'provide a SHORT and concise prescriptive feedback on the slide. Avoid euphemistic feedback, be critical. Use the following question to guide your feedback, check all checkpoints one after the other, even though not all need be applicable to this slide, consider wisely which one to recommend specifically. Do NOT just repeat the checkpoint descriptions. Additionally, if a checkpoint is applicable describe WHY, WHERE and HOW in the slide the checkpoints should be implemented: 1. Insert a pictogram. Pictogram insertion can intuitively convey the slide content. 2. Add a subheading. Subheading addition makes the audiences convenient to find information. 3. Emphasize words. Emphasizing important words enhances a part of a text to make it noticeable. 4. Emphasize areas. Emphasizing important areas of the slides helps focus attention. 5. Add T1 and T2. Slide title (T1) and slide messages (T2) help the audiences more easily understand the main content of the exposition. 6. Use the grid structure. A grid structure helps organize target and comparison items in rows and columns, creating a table-like format. 7. Itemize the text. Text itemization is good in terms of readability and organization. 8. Add a comment. Comments aid in audiences’ understanding. 9. Correct flow. Left-to-right top-to-bottom flow mimics human scanning methods. 10. MECE. Mutually exclusive and collectively exhaustive (MECE) objects should be aligned without omissions and duplication to make the slides convincing. 11. Consistent formatting. Look for and point out inconsistencies.' , 'answer': '**Slide Critique and Recommendations:**1. **Insert a Pictogram:**   - **Why:** The slide is text-heavy and lacks visual elements to make the content engaging and easier to understand. - **Where:** At the sections explaining concepts like ‘Generative AI’ or ‘ChatGPT’. - **How:** Use pictograms related to AI, machine learning, or chatbots. 2. **Add a Subheading:** - **Why:** Subheadings can break down the content into digestible sections. - **Where:** Subheadings can be added to sections like ‘Generative AI Platforms’, ‘Business Plan’, etc. - **How:** Use clear and descriptive subheadings such as ‘AI Applications’, ‘Business Strategy’, etc. 3. **Emphasize Words:** - **Why:** Important terms and key points are buried within dense paragraphs. - **Where:** Terms like ‘Generative AI’, ‘ChatGPT’, ‘prompt’, etc. - **How:** Use bold or italics to highlight these terms. 4. **Emphasize Areas:** - **Why:** Key areas need to stand out to capture attention quickly. - **Where:** In sections like ‘Stages of Development of Generative AI-Applications’ and ‘Benefits and Challenges of Using ChatGPT’. - **How:** Use colored boxes or borders around critical areas. 5. **Add T1 and T2:** - **Why:** The slide lacks a clear hierarchy, making it hard to follow. - **Where:** Title (T1) should be bold and larger, Slide messages (T2) should summarize key points. - **How:** Example - T1: ‘Generative AI Applications’, T2: ‘Various use cases showcasing the capabilities of generative AI’. 6. **Use the Grid Structure:** - **Why:** The slide lists several data points that would be clearer in a structured format. - **Where:** Sections like ‘Types of Generative AI and use cases’. - **How:** Create table-like grids for comparison and alignment. 7. **Itemize the Text:** - **Why:** Text blocks are unwieldy and hard to parse. - **Where:** Sections like ‘Business Plan’ and ‘Examples for the use of ChatGPT’. - **How:** Use bullet points to break down actions and steps. 8. **Add a comment:** - **Why:** Comments aid in audiences’ understanding.  - **Where:** Sections like ‘#\\n# Generative AI and ChatGPT’. - **How:** Add a comment in which you rephrase to help the audience understand quicker and easier. 9. **Correct Flow:** - **Why:** Content flow is currently disorganized and does not follow a logical progression. - **Where:** Rearrange sections so they logically follow each other (e.g., from basic definitions to practical applications). - **How:** Ensure the information follows a left-to-right or top-to-bottom flow. 10. **MECE:** - **Why:** The information needs to be more systematically categorized without overlaps or gaps. - **Where:** In all textual sections. - **How:**  Ensure each point falls under a distinct and necessary category, improving clarity and comprehension. 11. **Consistent Formatting:** - **Why:** Inconsistencies in font size, style, and indentation detract from professionalism. - **Where:** Across the entire slide, especially in headings and body texts. - **How:** Standardize font sizes and styles, and ensure uniform indentation. 12. **Citation Style:** - **Why:** Citations are scattered and not consistently formatted. - **Where:** At the end of sections or slides. - **How:** Use a consistent citation style like APA, and ensure uniform format for all references. Implementing these checkpoints will provide visual appeal, improve readability, and make the content easier to follow and understand.'}).with_inputs('question'),
]

gsm8k_devset = [
    Example({'question': "#\n# LLMs in Security: Today + What’s Coming\n\n# LLMs in Security: Today + What’s Coming\n\n# UC Berkeley CS294/194-196: Responsible GenAI and Decentralized Intelligence\n\nOctober 3rd, 2023\n\nLeo Meyerovich, Graphistry Inc.\n\n&GRAPHISTRY LOUIE.AI info@graphistry.com", 'gold_reasoning': 'provide a SHORT and concise prescriptive feedback on the slide. Avoid euphemistic feedback, be critical. Use the following question to guide your feedback, check all checkpoints one after the other, even though not all need be applicable to this slide, consider wisely which one to recommend specifically. Do NOT just repeat the checkpoint descriptions. Additionally, if a checkpoint is applicable describe WHY, WHERE and HOW in the slide the checkpoints should be implemented: 1. Insert a pictogram. Pictogram insertion can intuitively convey the slide content. 2. Add a subheading. Subheading addition makes the audiences convenient to find information. 3. Emphasize words. Emphasizing important words enhances a part of a text to make it noticeable. 4. Emphasize areas. Emphasizing important areas of the slides helps focus attention. 5. Add T1 and T2. Slide title (T1) and slide messages (T2) help the audiences more easily understand the main content of the exposition. 6. Use the grid structure. A grid structure helps organize target and comparison items in rows and columns, creating a table-like format. 7. Itemize the text. Text itemization is good in terms of readability and organization. 8. Add a comment. Comments aid in audiences’ understanding. 9. Correct flow. Left-to-right top-to-bottom flow mimics human scanning methods. 10. MECE. Mutually exclusive and collectively exhaustive (MECE) objects should be aligned without omissions and duplication to make the slides convincing. 11. Consistent formatting. Look for and point out inconsistencies.', 'answer': '**Slide Critique and Recommendations:**1. **Insert a Pictogram:**   - **Why:** A pictogram explains content visually. - **Where:** At the sections ‘LLMs in Security: Today + What’s Coming’ or ‘UC Berkeley CS294/194: Responsible GenAI and Decentralized Intelligence’, visual elements can make the slide more appealing. - **How:** Use pictograms related to AI, machine learning, or chatbots. 2. **Add a Subheading:** - **Why:** Subheadings can break down the content into digestible sections. - **Where:** Subheadings were already added successfully ‘UC Berkeley CS294/194: Responsible GenAI and Decentralized Intelligence’ 3. **Emphasize Words:** - **Why:** Important terms and key points are buried within dense paragraphs. - **Where:** Terms like ‘LLMs’, ‘Security’, ‘GenAI’, ‘Decentralized Intelligence’, etc. - **How:** Use bold or italics to highlight these terms. 4. **Emphasize Areas:** - **Why:** Key areas need to stand out to capture attention quickly. - **Where:** In sections like ‘LLMs in Security: Today + What’s Coming’. - **How:** Use colored boxes or borders around critical areas. 5. **Add T1 and T2:** - **Why:** The slide lacks a clear hierarchy, making it hard to follow. - **Where:** Title (T1) should be bold and larger, Slide messages (T2) should summarize key points. - **How:** Example - T1: ‘LLMs in Security: Today + What’s Coming’, T2: ‘UC Berkeley CS294/194: Responsible GenAI and Decentralized Intelligence’. 6. **Use the Grid Structure:** - **Why:** The slide lists several data points that would be clearer in a structured format. - **Where:** Sections like ‘Leo Meyerovich, Graphistry Inc.’, ‘GRAPHISTRY’, ‘LOUIE.AI’, ‘info@graphistry.com’. - **How:** Create grids for alignment. 7. **Itemize the Text:** - **Why:** Text blocks are unwieldy and hard to parse. - **Where:** This has been well done for the text on the whole slide. - **How:** Use bullet points to break down information in sections. 8. **Add a comment:** - **Why:** Comments aid in audiences’ understanding.  - **Where:** Sections like ‘LLMs in Security: Today + What’s Coming’ can use a comment or citation of someone to awaken interest into the topic. - **How:** Add a comment or citation in which you rephrase to help the audience understand quicker and easier. 9. **Correct Flow:** - **Why:** Content flow is currently disorganized and does not follow a logical progression. - **Where:** Rearrange sections so they logically follow each other (e.g., from basic definitions to practical applications). - **How:** Ensure the information follows a left-to-right or top-to-bottom flow. 10. **MECE:** - **Why:** The information needs to be more systematically categorized without overlaps or gaps. - **Where:** In all textual sections. - **How:**  Ensure each point falls under a distinct and necessary category, improving clarity and comprehension. 11. **Consistent Formatting:** - **Why:** Inconsistencies in font size, style, and indentation detract from professionalism. - **Where:** Across the entire slide, especially in headings and body texts. - **How:** Standardize font sizes and styles, and ensure uniform indentation. 12. **Citation Style:** - **Why:** Citations are scattered and not consistently formatted. - **Where:** At the end of sections or slides. - **How:** Use a consistent citation style like APA, and ensure uniform format for all references. Implementing these checkpoints will provide visual appeal, improve readability, and make the content easier to follow and understand.'}).with_inputs('question')
]
#gsm8k_trainset

Run our few examples and use them as a trainset.

In [14]:
gsm8k_trainset = [
    Example({'question': "#\\n# Generative AI and ChatGPT: Beyond the Hype\\n\\n# Generative AI and ChatGPT: Beyond the Hype\\n\\n# Prof. Dr. Jan Marco Leimeister\\n\\nFrom insight to impact.\\n---\\nBy 2025, generative AI is expected to account for 10% of all data produced (up from less than 1% in 2021).\\n\\nBy 2025, 30% of outbound marketing messages from large organizations will be synthetically generated (up from less than 2% in 2022).\\n\\nBy 2030, a major blockbuster film will be released with 90% of the film generated by AI (from 0% of such in 2022).\\n\\nSources: Gartner (2021) and Gartner (2023)\\n---\\n#\\n# Presentation Slide Deck\\n\\n# Agenda\\n\\n1. # Introduction to Generative AI\\n\\n- Definition and Development of a Generative AI\\n- Types of Generative AI, Use Cases and Platforms\\n2. # Latest Application: ChatGPT\\n\\n- Introduction and Use Cases for ChatGPT\\n- Constructing Effective Prompts\\n- Benefits and Challenges of Using ChatGPT\\n3. # Let‘s get to work!\\n4. # Summary\\n\\nChatGPT: Beyond the Hype | Prof. Dr. Jan Marco Leimeister (IWI-HSG)\\n---\\n# Introduction Prof. Dr. Jan Marco Leimeister\\n\\n- Professor and Director at the Institute of Information Management of the University of St.Gallen.\\n- Director at the ‘Wissenschaftlichen Zentrum für IT Gestaltung’ of the University of Kassel in Germany.\\n- Main research areas: Digital Business, Digital Transformation, Digital Service Management, Crowdsourcing, Digital Work and Collaboration Engineering.\\n- Internationally awarded for outstanding activities in research, teaching, and education.\\n- Repeatedly ranked among the top 1% of the best Researchers and Professors in business informatics.\\n---\\n#\\n# Presentation\\n\\n# Learning Goals\\n\\n’Generative AI and ChatGPT: Beyond the Hype’\\n\\n- You can name the key elements of Generative AI.\\n- You understand the ChatGPT application and its use cases.\\n- You can apply effective prompts for ChatGPT.\\n\\n&copy; Prof. Dr. Jan Marco Leimeister – Institut für Wirtschaftsinformatik (IWI-HSG) St.Gallen 5\\n---\\n# Introduction to Generative AI\\n---\\n#\\n# Adoption Rate: Bandwagon Effect at Work\\n\\nAdoption Rate: Bandwagon Effect at Work\\n\\nSource: (https://twitter.com/kylelf_) (Feb 16, 2023)\\n\\n© Prof. Dr. Jan Marco Leimeister – Institut für Wirtschaftsinformatik (IWI-HSG) St.Gallen\\n---\\n#\\n# Generative AI Definition\\n\\n# Definition of a Generative AI\\n\\n’Generative AI (GenAI) is a class of machine learning (ML) algorithms that can learn from content such as text, images, and audio in order to generate new content.’\\n\\nData:\\n\\n- Text: Training Transformer\\n- Images: Model (Neural Network)\\n- Speech: Add & Normalize Embedding\\n- Structured Data: 3D Signals\\n\\nTasks:\\n\\n- Question Answering\\n- Sentiment Analysis\\n- Info Extraction\\n- Image Captioning\\n- Object Recognition\\n\\nSources: Vaswani et al. (2017) and Sun et al. (2022)\\n\\n© Prof. Dr. Jan Marco Leimeister – Institut für Wirtschaftsinformatik (IWI-HSG) St.Gallen\\n---\\nDevelopment of a Generative AI\\n\\n- Collect demonstration data and train supervised policy: A prompt is sampled from our prompt dataset. The labeler demonstrates the desired output behavior. This data is used to fine-tune GPT-3,5 with supervised learning.\\n- Collect comparison data and train reward model using the PPO reinforcement learning algorithm: A new prompt is sampled from the dataset; outputs are sampled. The PPO model is initialized from the supervised policy. The policy generates an output; the labeler ranks the outputs from best to worst.\\n- High manual efforts required to train the supervised and reward model. Manual training is crowdsourced to the user. Select all frames with cars.\\n\\n$$\\\\text{Once upcnaumc} = 0.0-0$$\\n\\nThe reward model calculates reward for the output. This data is used to update the reward model and train our policy using PPO.\\n\\nSource: OpenAI (2023)\\n---\\n#\\n# Generative AI Applications Development Timeline\\n\\n# Stages of Development of Generative AI Applications\\n\\nFebruary 2023\\n\\n- Microsoft previews new Bing search engine\\n\\nNovember 2023\\n\\n", 'gold_reasoning': 'provide a SHORT and concise prescriptive feedback on the slide. Avoid euphemistic feedback, be critical. Use the following question to guide your feedback, check all checkpoints one after the other, even though not all need be applicable to this slide, consider wisely which one to recommend specifically. Do NOT just repeat the checkpoint descriptions. Additionally, if a checkpoint is applicable describe WHY, WHERE and HOW in the slide the checkpoints should be implemented: 1. Insert a pictogram. Pictogram insertion can intuitively convey the slide content. 2. Add a subheading. Subheading addition makes the audiences convenient to find information. 3. Emphasize words. Emphasizing important words enhances a part of a text to make it noticeable. 4. Emphasize areas. Emphasizing important areas of the slides helps focus attention. 5. Add T1 and T2. Slide title (T1) and slide messages (T2) help the audiences more easily understand the main content of the exposition. 6. Use the grid structure. A grid structure helps organize target and comparison items in rows and columns, creating a table-like format. 7. Itemize the text. Text itemization is good in terms of readability and organization. 8. Add a comment. Comments aid in audiences’ understanding. 9. Correct flow. Left-to-right top-to-bottom flow mimics human scanning methods. 10. MECE. Mutually exclusive and collectively exhaustive (MECE) objects should be aligned without omissions and duplication to make the slides convincing. 11. Consistent formatting. Look for and point out inconsistencies.' , 'answer': '**Slide Critique and Recommendations:**1. **Insert a Pictogram:**   - **Why:** The slide is text-heavy and lacks visual elements to make the content engaging and easier to understand. - **Where:** At the sections explaining concepts like ‘Generative AI’ or ‘ChatGPT’. - **How:** Use pictograms related to AI, machine learning, or chatbots. 2. **Add a Subheading:** - **Why:** Subheadings can break down the content into digestible sections. - **Where:** Subheadings can be added to sections like ‘Generative AI Platforms’, ‘Business Plan’, etc. - **How:** Use clear and descriptive subheadings such as ‘AI Applications’, ‘Business Strategy’, etc. 3. **Emphasize Words:** - **Why:** Important terms and key points are buried within dense paragraphs. - **Where:** Terms like ‘Generative AI’, ‘ChatGPT’, ‘prompt’, etc. - **How:** Use bold or italics to highlight these terms. 4. **Emphasize Areas:** - **Why:** Key areas need to stand out to capture attention quickly. - **Where:** In sections like ‘Stages of Development of Generative AI-Applications’ and ‘Benefits and Challenges of Using ChatGPT’. - **How:** Use colored boxes or borders around critical areas. 5. **Add T1 and T2:** - **Why:** The slide lacks a clear hierarchy, making it hard to follow. - **Where:** Title (T1) should be bold and larger, Slide messages (T2) should summarize key points. - **How:** Example - T1: ‘Generative AI Applications’, T2: ‘Various use cases showcasing the capabilities of generative AI’. 6. **Use the Grid Structure:** - **Why:** The slide lists several data points that would be clearer in a structured format. - **Where:** Sections like ‘Types of Generative AI and use cases’. - **How:** Create table-like grids for comparison and alignment. 7. **Itemize the Text:** - **Why:** Text blocks are unwieldy and hard to parse. - **Where:** Sections like ‘Business Plan’ and ‘Examples for the use of ChatGPT’. - **How:** Use bullet points to break down actions and steps. 8. **Add a comment:** - **Why:** Comments aid in audiences’ understanding.  - **Where:** Sections like ‘#\\n# Generative AI and ChatGPT’. - **How:** Add a comment in which you rephrase to help the audience understand quicker and easier. 9. **Correct Flow:** - **Why:** Content flow is currently disorganized and does not follow a logical progression. - **Where:** Rearrange sections so they logically follow each other (e.g., from basic definitions to practical applications). - **How:** Ensure the information follows a left-to-right or top-to-bottom flow. 10. **MECE:** - **Why:** The information needs to be more systematically categorized without overlaps or gaps. - **Where:** In all textual sections. - **How:**  Ensure each point falls under a distinct and necessary category, improving clarity and comprehension. 11. **Consistent Formatting:** - **Why:** Inconsistencies in font size, style, and indentation detract from professionalism. - **Where:** Across the entire slide, especially in headings and body texts. - **How:** Standardize font sizes and styles, and ensure uniform indentation. 12. **Citation Style:** - **Why:** Citations are scattered and not consistently formatted. - **Where:** At the end of sections or slides. - **How:** Use a consistent citation style like APA, and ensure uniform format for all references. Implementing these checkpoints will provide visual appeal, improve readability, and make the content easier to follow and understand.'}).with_inputs('question'),
    Example({'question': "#\\n# Generative AI and ChatGPT: Beyond the Hype\\n\\n# Generative AI and ChatGPT: Beyond the Hype\\n\\n# Prof. Dr. Jan Marco Leimeister\\n\\nFrom insight to impact.\\n---\\nBy 2025, generative AI is expected to account for 10% of all data produced (up from less than 1% in 2021).\\n\\nBy 2025, 30% of outbound marketing messages from large organizations will be synthetically generated (up from less than 2% in 2022).\\n\\nBy 2030, a major blockbuster film will be released with 90% of the film generated by AI (from 0% of such in 2022).\\n\\nSources: Gartner (2021) and Gartner (2023)\\n---\\n#\\n# Presentation Slide Deck\\n\\n# Agenda\\n\\n1. # Introduction to Generative AI\\n\\n- Definition and Development of a Generative AI\\n- Types of Generative AI, Use Cases and Platforms\\n2. # Latest Application: ChatGPT\\n\\n- Introduction and Use Cases for ChatGPT\\n- Constructing Effective Prompts\\n- Benefits and Challenges of Using ChatGPT\\n3. # Let‘s get to work!\\n4. # Summary\\n\\nChatGPT: Beyond the Hype | Prof. Dr. Jan Marco Leimeister (IWI-HSG)\\n---\\n# Introduction Prof. Dr. Jan Marco Leimeister\\n\\n- Professor and Director at the Institute of Information Management of the University of St.Gallen.\\n- Director at the ‘Wissenschaftlichen Zentrum für IT Gestaltung’ of the University of Kassel in Germany.\\n- Main research areas: Digital Business, Digital Transformation, Digital Service Management, Crowdsourcing, Digital Work and Collaboration Engineering.\\n- Internationally awarded for outstanding activities in research, teaching, and education.\\n- Repeatedly ranked among the top 1% of the best Researchers and Professors in business informatics.\\n---\\n#\\n# Presentation\\n\\n# Learning Goals\\n\\n’Generative AI and ChatGPT: Beyond the Hype’\\n\\n- You can name the key elements of Generative AI.\\n- You understand the ChatGPT application and its use cases.\\n- You can apply effective prompts for ChatGPT.\\n\\n&copy; Prof. Dr. Jan Marco Leimeister – Institut für Wirtschaftsinformatik (IWI-HSG) St.Gallen 5\\n---\\n# Introduction to Generative AI\\n---\\n#\\n# Adoption Rate: Bandwagon Effect at Work\\n\\nAdoption Rate: Bandwagon Effect at Work\\n\\nSource: (https://twitter.com/kylelf_) (Feb 16, 2023)\\n\\n© Prof. Dr. Jan Marco Leimeister – Institut für Wirtschaftsinformatik (IWI-HSG) St.Gallen\\n---\\n#\\n# Generative AI Definition\\n\\n# Definition of a Generative AI\\n\\n’Generative AI (GenAI) is a class of machine learning (ML) algorithms that can learn from content such as text, images, and audio in order to generate new content.’\\n\\nData:\\n\\n- Text: Training Transformer\\n- Images: Model (Neural Network)\\n- Speech: Add & Normalize Embedding\\n- Structured Data: 3D Signals\\n\\nTasks:\\n\\n- Question Answering\\n- Sentiment Analysis\\n- Info Extraction\\n- Image Captioning\\n- Object Recognition\\n\\nSources: Vaswani et al. (2017) and Sun et al. (2022)\\n\\n© Prof. Dr. Jan Marco Leimeister – Institut für Wirtschaftsinformatik (IWI-HSG) St.Gallen\\n---\\nDevelopment of a Generative AI\\n\\n- Collect demonstration data and train supervised policy: A prompt is sampled from our prompt dataset. The labeler demonstrates the desired output behavior. This data is used to fine-tune GPT-3,5 with supervised learning.\\n- Collect comparison data and train reward model using the PPO reinforcement learning algorithm: A new prompt is sampled from the dataset; outputs are sampled. The PPO model is initialized from the supervised policy. The policy generates an output; the labeler ranks the outputs from best to worst.\\n- High manual efforts required to train the supervised and reward model. Manual training is crowdsourced to the user. Select all frames with cars.\\n\\n$$\\\\text{Once upcnaumc} = 0.0-0$$\\n\\nThe reward model calculates reward for the output. This data is used to update the reward model and train our policy using PPO.\\n\\nSource: OpenAI (2023)\\n---\\n#\\n# Generative AI Applications Development Timeline\\n\\n# Stages of Development of Generative AI Applications\\n\\nFebruary 2023\\n\\n- Microsoft previews new Bing search engine\\n\\nNovember 2023\\n\\n", 'gold_reasoning': 'provide a SHORT and concise prescriptive feedback on the slide. Avoid euphemistic feedback, be critical. Use the following question to guide your feedback, check all checkpoints one after the other, even though not all need be applicable to this slide, consider wisely which one to recommend specifically. Do NOT just repeat the checkpoint descriptions. Additionally, if a checkpoint is applicable describe WHY, WHERE and HOW in the slide the checkpoints should be implemented: 1. Insert a pictogram. Pictogram insertion can intuitively convey the slide content. 2. Add a subheading. Subheading addition makes the audiences convenient to find information. 3. Emphasize words. Emphasizing important words enhances a part of a text to make it noticeable. 4. Emphasize areas. Emphasizing important areas of the slides helps focus attention. 5. Add T1 and T2. Slide title (T1) and slide messages (T2) help the audiences more easily understand the main content of the exposition. 6. Use the grid structure. A grid structure helps organize target and comparison items in rows and columns, creating a table-like format. 7. Itemize the text. Text itemization is good in terms of readability and organization. 8. Add a comment. Comments aid in audiences’ understanding. 9. Correct flow. Left-to-right top-to-bottom flow mimics human scanning methods. 10. MECE. Mutually exclusive and collectively exhaustive (MECE) objects should be aligned without omissions and duplication to make the slides convincing. 11. Consistent formatting. Look for and point out inconsistencies.' , 'answer': '**Slide Critique and Recommendations:**1. **Insert a Pictogram:**   - **Why:** The slide is text-heavy and lacks visual elements to make the content engaging and easier to understand. - **Where:** At the sections explaining concepts like ‘Generative AI’ or ‘ChatGPT’. - **How:** Use pictograms related to AI, machine learning, or chatbots. 2. **Add a Subheading:** - **Why:** Subheadings can break down the content into digestible sections. - **Where:** Subheadings can be added to sections like ‘Generative AI Platforms’, ‘Business Plan’, etc. - **How:** Use clear and descriptive subheadings such as ‘AI Applications’, ‘Business Strategy’, etc. 3. **Emphasize Words:** - **Why:** Important terms and key points are buried within dense paragraphs. - **Where:** Terms like ‘Generative AI’, ‘ChatGPT’, ‘prompt’, etc. - **How:** Use bold or italics to highlight these terms. 4. **Emphasize Areas:** - **Why:** Key areas need to stand out to capture attention quickly. - **Where:** In sections like ‘Stages of Development of Generative AI-Applications’ and ‘Benefits and Challenges of Using ChatGPT’. - **How:** Use colored boxes or borders around critical areas. 5. **Add T1 and T2:** - **Why:** The slide lacks a clear hierarchy, making it hard to follow. - **Where:** Title (T1) should be bold and larger, Slide messages (T2) should summarize key points. - **How:** Example - T1: ‘Generative AI Applications’, T2: ‘Various use cases showcasing the capabilities of generative AI’. 6. **Use the Grid Structure:** - **Why:** The slide lists several data points that would be clearer in a structured format. - **Where:** Sections like ‘Types of Generative AI and use cases’. - **How:** Create table-like grids for comparison and alignment. 7. **Itemize the Text:** - **Why:** Text blocks are unwieldy and hard to parse. - **Where:** Sections like ‘Business Plan’ and ‘Examples for the use of ChatGPT’. - **How:** Use bullet points to break down actions and steps. 8. **Add a comment:** - **Why:** Comments aid in audiences’ understanding.  - **Where:** Sections like ‘#\\n# Generative AI and ChatGPT’. - **How:** Add a comment in which you rephrase to help the audience understand quicker and easier. 9. **Correct Flow:** - **Why:** Content flow is currently disorganized and does not follow a logical progression. - **Where:** Rearrange sections so they logically follow each other (e.g., from basic definitions to practical applications). - **How:** Ensure the information follows a left-to-right or top-to-bottom flow. 10. **MECE:** - **Why:** The information needs to be more systematically categorized without overlaps or gaps. - **Where:** In all textual sections. - **How:**  Ensure each point falls under a distinct and necessary category, improving clarity and comprehension. 11. **Consistent Formatting:** - **Why:** Inconsistencies in font size, style, and indentation detract from professionalism. - **Where:** Across the entire slide, especially in headings and body texts. - **How:** Standardize font sizes and styles, and ensure uniform indentation. 12. **Citation Style:** - **Why:** Citations are scattered and not consistently formatted. - **Where:** At the end of sections or slides. - **How:** Use a consistent citation style like APA, and ensure uniform format for all references. Implementing these checkpoints will provide visual appeal, improve readability, and make the content easier to follow and understand.'}).with_inputs('question')
]

gsm8k_devset = [
    Example({'question': "#\n# LLMs in Security: Today + What’s Coming\n\n# LLMs in Security: Today + What’s Coming\n\n# UC Berkeley CS294/194-196: Responsible GenAI and Decentralized Intelligence\n\nOctober 3rd, 2023\n\nLeo Meyerovich, Graphistry Inc.\n\n&GRAPHISTRY LOUIE.AI info@graphistry.com", 'gold_reasoning': 'provide a SHORT and concise prescriptive feedback on the slide. Avoid euphemistic feedback, be critical. Use the following question to guide your feedback, check all checkpoints one after the other, even though not all need be applicable to this slide, consider wisely which one to recommend specifically. Do NOT just repeat the checkpoint descriptions. Additionally, if a checkpoint is applicable describe WHY, WHERE and HOW in the slide the checkpoints should be implemented: 1. Insert a pictogram. Pictogram insertion can intuitively convey the slide content. 2. Add a subheading. Subheading addition makes the audiences convenient to find information. 3. Emphasize words. Emphasizing important words enhances a part of a text to make it noticeable. 4. Emphasize areas. Emphasizing important areas of the slides helps focus attention. 5. Add T1 and T2. Slide title (T1) and slide messages (T2) help the audiences more easily understand the main content of the exposition. 6. Use the grid structure. A grid structure helps organize target and comparison items in rows and columns, creating a table-like format. 7. Itemize the text. Text itemization is good in terms of readability and organization. 8. Add a comment. Comments aid in audiences’ understanding. 9. Correct flow. Left-to-right top-to-bottom flow mimics human scanning methods. 10. MECE. Mutually exclusive and collectively exhaustive (MECE) objects should be aligned without omissions and duplication to make the slides convincing. 11. Consistent formatting. Look for and point out inconsistencies.', 'answer': '**Slide Critique and Recommendations:**1. **Insert a Pictogram:**   - **Why:** A pictogram explains content visually. - **Where:** At the sections ‘LLMs in Security: Today + What’s Coming’ or ‘UC Berkeley CS294/194: Responsible GenAI and Decentralized Intelligence’, visual elements can make the slide more appealing. - **How:** Use pictograms related to AI, machine learning, or chatbots. 2. **Add a Subheading:** - **Why:** Subheadings can break down the content into digestible sections. - **Where:** Subheadings were already added successfully ‘UC Berkeley CS294/194: Responsible GenAI and Decentralized Intelligence’ 3. **Emphasize Words:** - **Why:** Important terms and key points are buried within dense paragraphs. - **Where:** Terms like ‘LLMs’, ‘Security’, ‘GenAI’, ‘Decentralized Intelligence’, etc. - **How:** Use bold or italics to highlight these terms. 4. **Emphasize Areas:** - **Why:** Key areas need to stand out to capture attention quickly. - **Where:** In sections like ‘LLMs in Security: Today + What’s Coming’. - **How:** Use colored boxes or borders around critical areas. 5. **Add T1 and T2:** - **Why:** The slide lacks a clear hierarchy, making it hard to follow. - **Where:** Title (T1) should be bold and larger, Slide messages (T2) should summarize key points. - **How:** Example - T1: ‘LLMs in Security: Today + What’s Coming’, T2: ‘UC Berkeley CS294/194: Responsible GenAI and Decentralized Intelligence’. 6. **Use the Grid Structure:** - **Why:** The slide lists several data points that would be clearer in a structured format. - **Where:** Sections like ‘Leo Meyerovich, Graphistry Inc.’, ‘GRAPHISTRY’, ‘LOUIE.AI’, ‘info@graphistry.com’. - **How:** Create grids for alignment. 7. **Itemize the Text:** - **Why:** Text blocks are unwieldy and hard to parse. - **Where:** This has been well done for the text on the whole slide. - **How:** Use bullet points to break down information in sections. 8. **Add a comment:** - **Why:** Comments aid in audiences’ understanding.  - **Where:** Sections like ‘LLMs in Security: Today + What’s Coming’ can use a comment or citation of someone to awaken interest into the topic. - **How:** Add a comment or citation in which you rephrase to help the audience understand quicker and easier. 9. **Correct Flow:** - **Why:** Content flow is currently disorganized and does not follow a logical progression. - **Where:** Rearrange sections so they logically follow each other (e.g., from basic definitions to practical applications). - **How:** Ensure the information follows a left-to-right or top-to-bottom flow. 10. **MECE:** - **Why:** The information needs to be more systematically categorized without overlaps or gaps. - **Where:** In all textual sections. - **How:**  Ensure each point falls under a distinct and necessary category, improving clarity and comprehension. 11. **Consistent Formatting:** - **Why:** Inconsistencies in font size, style, and indentation detract from professionalism. - **Where:** Across the entire slide, especially in headings and body texts. - **How:** Standardize font sizes and styles, and ensure uniform indentation. 12. **Citation Style:** - **Why:** Citations are scattered and not consistently formatted. - **Where:** At the end of sections or slides. - **How:** Use a consistent citation style like APA, and ensure uniform format for all references. Implementing these checkpoints will provide visual appeal, improve readability, and make the content easier to follow and understand.'}).with_inputs('question')
]

In [15]:
# Load math questions from the GSM8K dataset
gsm8k = GSM8K()
# gsm8k_trainset, gsm8k_devset = gsm8k.train[:10], gsm8k.dev[:10]
# print(gsm8k.train[:10])

100%|██████████| 1319/1319 [00:00<00:00, 68080.47it/s]


In [23]:
print(gsm8k_trainset)
print(gsm8k_devset)

[Example({'question': '#\\n# Generative AI and ChatGPT: Beyond the Hype\\n\\n# Generative AI and ChatGPT: Beyond the Hype\\n\\n# Prof. Dr. Jan Marco Leimeister\\n\\nFrom insight to impact.\\n---\\nBy 2025, generative AI is expected to account for 10% of all data produced (up from less than 1% in 2021).\\n\\nBy 2025, 30% of outbound marketing messages from large organizations will be synthetically generated (up from less than 2% in 2022).\\n\\nBy 2030, a major blockbuster film will be released with 90% of the film generated by AI (from 0% of such in 2022).\\n\\nSources: Gartner (2021) and Gartner (2023)\\n---\\n#\\n# Presentation Slide Deck\\n\\n# Agenda\\n\\n1. # Introduction to Generative AI\\n\\n- Definition and Development of a Generative AI\\n- Types of Generative AI, Use Cases and Platforms\\n2. # Latest Application: ChatGPT\\n\\n- Introduction and Use Cases for ChatGPT\\n- Constructing Effective Prompts\\n- Benefits and Challenges of Using ChatGPT\\n3. # Let‘s get to work!\\n4. #

Code our Chain-Of-Thought logic into a method using information from dspy.Module and equip it with the function to forward a question.

In [17]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought("question -> answer")

    def forward(self, question):
        return self.prog(question=question)

gsm8k = GSM8K()

100%|██████████| 1319/1319 [00:00<00:00, 68119.03it/s]


Set up our optimizer and apply it along our LLM-calling program to be optimized on a given gsm8k_metric.


In [18]:
from dspy.teleprompt import BootstrapFewShot
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
import os
# Set up the optimizer: we want to "bootstrap" (i.e., self-generate) 4-shot examples of our CoT program.
config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)
# Optimize! Use the `gsm8k_metric` here. In general, the metric is going to tell the optimizer how well it's doing.
teleprompter = BootstrapFewShot(metric=gsm8k_metric, **config)
optimized_cot = teleprompter.compile(CoT(), trainset=gsm8k_trainset)

100%|██████████| 2/2 [00:00<00:00, 1112.99it/s]

Bootstrapped 0 full traces after 2 examples in round 0.


For the sake of completeness, we are doing the evaluation step here. However, this is not necessary here, as the evaluation on a gsm8k dataset does not match our use case.

In [19]:
from dspy.evaluate import Evaluate

# Set up the evaluator, which can be used multiple times.
evaluate = Evaluate(devset=gsm8k_devset, metric=gsm8k_metric, num_threads=1, display_progress=True, display_table=0)

# Evaluate our `optimized_cot` program.
evaluate(optimized_cot)

Average Metric: 0.0 / 1  (0.0): 100%|██████████| 1/1 [00:00<00:00, 190.95it/s]

Error for example in dev set: 		 The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable
Average Metric: 0.0 / 1  (0.0%)


0.0

In the following, we can see that the output of the LLM matches our expectations as all checkpoints have been processed and the response addresses the where, how and why in every checkpoint.

In [21]:
turbo.inspect_history(n=1)





Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: #\n# Generative AI and ChatGPT: Beyond the Hype\n\n# Generative AI and ChatGPT: Beyond the Hype\n\n# Prof. Dr. Jan Marco Leimeister\n\nFrom insight to impact.\n---\nBy 2025, generative AI is expected to account for 10% of all data produced (up from less than 1% in 2021).\n\nBy 2025, 30% of outbound marketing messages from large organizations will be synthetically generated (up from less than 2% in 2022).\n\nBy 2030, a major blockbuster film will be released with 90% of the film generated by AI (from 0% of such in 2022).\n\nSources: Gartner (2021) and Gartner (2023)\n---\n#\n# Presentation Slide Deck\n\n# Agenda\n\n1. # Introduction to Generative AI\n\n- Definition and Development of a Generative AI\n- Types of Generative AI, Use Cases and Platforms\n2. # Lat